# Creating a Simple Agent with Tracing

In [1]:
import dotenv
import os

from openai import OpenAI

dotenv.load_dotenv()

if not os.environ.get("OPENAI_API_KEY"):
    print(
        """Error: OPENAI_API_KEY environment variable not set. Please copy the .env.template file as .env and fill it in.
    
    You can execute these commands in the terminal to get started:
    cp .env.template .env
    code .env
    """
    )

# Test OpenAI Access
print(
    OpenAI()
    .responses.create(
        model=os.environ["OPENAI_DEFAULT_MODEL"], input="Say: We are up and running!"
    )
    .output_text
)

We are up and running!


In [2]:
from agents import Agent, Runner, trace
from openai.types.responses import ResponseTextDeltaEvent

Create a simple Nutrition Assistant Agent

In [4]:
nutrition_agent = Agent(
    name="Nutrition Assistant",
    instructions="""
    You are a helpful assitant giving out nutritional advice.
    You give concise answers
    """
)

Let's execute the Agent:

In [5]:
with trace("Simple Nutrition Agent"):
    result = await Runner.run(nutrition_agent, "How healthy are bananas?")

print(result)

RunResult:
- Last agent: Agent(name="Nutrition Assistant", ...)
- Final output (str):
    Bananas are healthy when eaten as part of a balanced diet.
    
    Key positives:
    - Good source of potassium (helps blood pressure) and vitamin B6
    - Vitamin C, dietary fiber, and resistant starch (especially when green)
    - Low in fat and calories for a fruit
    
    Considerations:
    - Contain natural sugars; portion if you’re watching blood sugar or calories
    - modest protein and fat
    
    Practical tips:
    - One medium banana (~105 kcal) is a reasonable serving
    - Pair with protein or healthy fats for balanced meals
    
    Overall: they’re nutritious, versatile, and generally a good snack.
- 2 new item(s)
- 1 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)


Streaming the answer to the screen, token by token

In [6]:
response_stream = Runner.run_streamed(nutrition_agent, "How healthy are bananas?")

async for event in response_stream.stream_events():
    if event.type == "raw_response_event" and isinstance(
        event.data, ResponseTextDeltaEvent
    ):
        print(event.data.delta, end="", flush=True)

Generally healthy as part of a balanced diet.

Nutrients in a medium banana (~118 g):
- Potassium: about 420–430 mg (good for blood pressure)
- Vitamin B6: ~20–25% of daily needs
- Vitamin C: ~10% of daily needs
- Fiber: ~3 g (supports digestion)
- Natural sugars: ~14 g

Benefits:
- May support heart health, energy, and satiety
- Convenient pre-/post-workout snack

Notes:
- Ripeness affects sugar content (fewer sugars with greener; more with ripe)
- Pair with protein or fat for better satiety
- Moderation if you’re watching carbs or sugar intake

Overall: a healthy choice for most people when eaten in moderation.

_Good Job!_